In [4]:
!pip uninstall numpy scipy
!pip install numpy scipy
!pip uninstall xgboost
!pip install xgboost
!pip install biopython


In [ ]:
#!pip install git+https://github.com/exobyte-labs/betamark.git

In [ ]:
from Bio import motifs
from Bio.motifs import jaspar

with open("JASPAR2024_CORE_redundant_pfms_jaspar.txt") as handle:
    motif_list = motifs.parse(handle, "jaspar")

all_consensus = []
for motif in motif_list:
    print("Consensus sequence:", motif.consensus)
    all_consensus.append(str(motif.consensus))

In [3]:
import xgboost as xgb

# Create an instance of the XGBClassifier (or your specific model type)
loaded_model = xgb.XGBClassifier()

# Load the model from the file
loaded_model.load_model('genomic/xgbclass_submission.json')

ImportError: DLL load failed while importing _fblas: %1 is not a valid Win32 application.

In [ ]:
from betamark import ocr
import pandas as pd
import numpy as np

def placeholder(x):
    """
    Params:
    -------
    x: string representing a genomic sequence

    Returns:
    --------
    y_pred: int where 0 is negative (not an OCR) or 1 (is an OCR)
    """
    
    # extract as many features as we can think of
    df = pd.DataFrame()

    # number of nucleotides
    df['length'] = x.len()

    # % of A, C, T and G's (and N's)
    df['n_A'] = x.count("A") / df['length']
    df['n_C'] = x.count("C") / df['length']
    df['n_T'] = x.count("T") / df['length']
    df['n_G'] = x.count("G") / df['length']
    #df['n_N'] = x.count("N") / df['length'] # this is not needed, it is implied by the other columns

    df['cpg_ratio'] = np.where(
        (x.count('C') * x.count('G')) > 0,
        (x.count('CG') * df['length']) / (x.count('C') * x.count('G')),
        0
    )

    df['motif_count'] = sum(x.count(motif) for motif in all_consensus)

    # does it contain a stop codon within it? (TAG, TAA, TGA) ?
    df['contains_stop'] = x.contains('TAG|TAA|TGA').astype(int)

    # does it have a start codon within in (AUG) ?
    #df['contains_start'] = df['seq'].str.contains('ATG').astype(int)

    # GC content\
    df['GC_content'] = (df['seq'].str.count("G") + df['seq'].str.count("C")) / df['length']

    # motifs between positive sequences? (k-mers?)
    print(df.head(1))
    
    result = loaded_model.predict(df)
    print(result)
    return(result)

ocr.run_eval(user_func=placeholder)